# Tutorial4: visualize recovery results

---
This tutorial demonstrates visualize the 3D / 4D recovery results. \
By sampling a trained neural network at regular grid intervals we can visualize the recovered 3D emission.

In [86]:
bhnerf.constants.sgra_mass

<Quantity 8.2598546e+36 kg>

In [1]:
import bhnerf
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from jax import numpy as jnp
from flax.training import checkpoints
from astropy import units
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-l7hxp8i9 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-07-05 18:53:05.854196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [81]:
checkpoint_dir = '../checkpoints/tutorial3/test_no_g.t.2022-07-05.18:55:19/'
predictor = bhnerf.network.NeRF_Predictor()
state = checkpoints.restore_checkpoint(checkpoint_dir, None)

In [18]:
"""
Define a VolumeVisualizer object
"""
resolution = 128
visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

In [19]:
"""
Define the position of the camera and sample/integrate emission according to the camera rays.
View the estimated emission at t=0 from multiple angles. 
Note that jit is useful for acceleration however the initial rendering will be slow due to run time compilation.
"""
images = []
bh_radius = 2.0
norm_const = 0.02
rmax = 8.0

for azimuth in tqdm(np.linspace(0.0, 360, 6)):
    visualizer.set_view(radius=32.5, azimuth=azimuth, zenith=np.pi/3)
    emission = predictor.apply({'params': state['params']}, 0.0, units.hr, visualizer.coords, 0.0, 0.0, 0.0, 0.0)
    emission = bhnerf.emission.fill_unsupervised_emission(emission, visualizer.coords, rmin=bh_radius, rmax=rmax)
    images.append(visualizer.render(emission, facewidth=rmax, jit=True, norm_const=norm_const, bh_radius=bh_radius))

  0%|          | 0/6 [00:00<?, ?it/s]

In [84]:
images = []
bh_radius = 2.0
norm_const = 0.02
rmax = 8.0
azimuth = 0.0

visualizer.set_view(radius=27.5, azimuth=azimuth, zenith=np.pi/3)
emission = predictor.apply({'params': state['params']}, 0.0, units.hr, visualizer.coords, 0.0, 0.0, 0.0, 0.0)
emission = bhnerf.emission.fill_unsupervised_emission(emission, visualizer.coords, rmin=bh_radius)
image = visualizer.render(emission, facewidth=2*rmax, jit=True, norm_const=norm_const, bh_radius=bh_radius)

In [85]:
%matplotlib widget
plt.imshow(image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
%matplotlib widget
fig, axes = plt.subplots(1, 6, figsize=(10,2))
for ax, image in zip(axes, images):
    ax.imshow(image);
    ax.axis('off');
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …